In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean-previous-applicationcsv/clean_previous_application.csv
/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/clean-credit-card-balance/clean_credit_card_balance.csv


# 1. Reading data

In [2]:
installments_payments = pd.read_csv("../input/home-credit-default-risk/installments_payments.csv")
pos_cash = pd.read_csv("../input/home-credit-default-risk/POS_CASH_balance.csv")
application_train = pd.read_csv("../input/home-credit-default-risk/application_train.csv")
previous_application = pd.read_csv("../input/clean-previous-applicationcsv/clean_previous_application.csv", index_col=["Unnamed: 0"])

In [3]:
previous_application

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,log_AMT_CREDIT,log_AMT_DOWN_PAYMENT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,...,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495.0,271877.0,Consumer loans,9.749520,0.000000,SATURDAY,15.0,Y,1.0,0.000000,...,35.0,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-37.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2819243.0,176158.0,Cash loans,13.062170,NaN,MONDAY,7.0,Y,1.0,NaN,...,-1.0,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-177.0,1.0
4,1784265.0,202054.0,Cash loans,12.909309,NaN,THURSDAY,9.0,Y,1.0,NaN,...,-1.0,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464.0,352015.0,Consumer loans,12.648837,0.000000,WEDNESDAY,12.0,Y,1.0,0.000000,...,43.0,Furniture,30.0,low_normal,POS industry with interest,365243.0,-508.0,362.0,-351.0,0.0
1670210,2357031.0,334635.0,Consumer loans,11.071198,10.283669,TUESDAY,15.0,Y,1.0,0.340554,...,43.0,Furniture,12.0,middle,POS industry with interest,365243.0,-1604.0,-1274.0,-1297.0,0.0
1670211,2659632.0,249544.0,Consumer loans,11.537857,9.261651,MONDAY,12.0,Y,1.0,0.101401,...,1370.0,Consumer electronics,10.0,low_normal,POS household with interest,365243.0,-1457.0,-1187.0,-1181.0,0.0
1670212,2785582.0,400317.0,Cash loans,12.164631,NaN,WEDNESDAY,9.0,Y,1.0,NaN,...,-1.0,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-1155.0,-825.0,-817.0,1.0


# 2. Merge credit card balance and installment

# 2.1 Inspect credit card balance

In [4]:
credit_card_balance = pd.read_csv("../input/clean-credit-card-balance/clean_credit_card_balance.csv", index_col=["Unnamed: 0"])

In [5]:
credit_card_balance.groupby(["SK_ID_PREV"]).nunique()["SK_ID_CURR"].max()

1

In [6]:
credit_card_balance[credit_card_balance["SK_ID_PREV"] == 2843461][["CNT_INSTALMENT_MATURE_CUM", "MONTHS_BALANCE"]].sort_values("MONTHS_BALANCE")

,CNT_INSTALMENT_MATURE_CUM,MONTHS_BALANCE
3525644,NaN,-73
694721,1.0,-72
309242,2.0,-71
2307409,3.0,-70
319051,4.0,-69
...,...,...
310934,47.0,-5
1108652,48.0,-4
1536831,49.0,-3
228819,50.0,-2


In [7]:
description = pd.read_csv("../input/home-credit-default-risk/HomeCredit_columns_description.csv", encoding= "ISO-8859-1")
description = description[description["Table"] == "credit_card_balance.csv"]
description
def search_description(name):
    return description[description["Row"] == name].values

In [8]:
search_description("CNT_INSTALMENT_MATURE_CUM")

array([[172, 'credit_card_balance.csv', 'CNT_INSTALMENT_MATURE_CUM',
        'Number of paid installments on the previous credit', nan]],
      dtype=object)

Maximum of count values is the total number of paid installments, total number of drawings,... of each previous application

In [9]:
credit_card_balance[credit_card_balance.duplicated(subset = ["SK_ID_PREV"])]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_PAYMENT_CURRENT,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
4960,1310382,129897,-16,0.000,0,0.0,0.0,0.0,239.265,0.0,0,0.0,9.0,Active,0,0
4976,2162384,136037,-18,0.000,135000,0.0,NaN,NaN,NaN,NaN,0,NaN,0.0,Active,0,0
4987,1668231,220058,-22,0.000,45000,0.0,NaN,NaN,NaN,NaN,0,NaN,0.0,Active,0,0
5005,1270749,353826,-20,0.000,67500,0.0,0.0,0.0,0.000,0.0,0,0.0,26.0,Completed,0,0
5014,2653174,310463,-22,0.000,180000,0.0,NaN,NaN,NaN,NaN,0,NaN,0.0,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,0.0,NaN,NaN,NaN,NaN,0,NaN,0.0,Active,0,0
3840308,1714892,347207,-9,0.000,45000,0.0,0.0,0.0,1879.110,0.0,0,0.0,23.0,Active,0,0
3840309,1302323,215757,-9,275784.975,585000,270000.0,0.0,0.0,375750.000,2.0,2,0.0,18.0,Active,0,0
3840310,1624872,430337,-10,0.000,450000,0.0,NaN,NaN,NaN,NaN,0,NaN,0.0,Active,0,0


In [10]:
credit_card_balance_amt = credit_card_balance.filter(regex = "AMT_")
credit_card_balance_amt["SK_ID_PREV"] = credit_card_balance["SK_ID_PREV"]
credit_grouped_amt = credit_card_balance_amt.groupby("SK_ID_PREV")
credit_grouped_amt_mean = credit_grouped_amt.mean()
credit_grouped_amt_mean = credit_grouped_amt_mean.add_suffix("_avg")
credit_grouped_amt_mean.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,AMT_BALANCE_avg,AMT_CREDIT_LIMIT_ACTUAL_avg,AMT_DRAWINGS_CURRENT_avg,AMT_DRAWINGS_OTHER_CURRENT_avg,AMT_DRAWINGS_POS_CURRENT_avg,AMT_PAYMENT_CURRENT_avg
SK_ID_PREV,,,,,,
1000018,74946.285000,81000.000000,29478.996000,0.0,24078.996000,5541.750000
1000030,55991.064375,81562.500000,17257.438125,0.0,19079.929286,6188.631429
1000031,52394.439375,149625.000000,28959.615000,0.0,23527.218462,29543.257500
1000035,0.000000,225000.000000,0.000000,NaN,NaN,NaN
1000077,0.000000,94090.909091,0.000000,NaN,NaN,NaN


In [11]:
credit_card_balance_cnt = credit_card_balance.filter(regex = "CNT_")
credit_card_balance_cnt["SK_ID_PREV"] = credit_card_balance["SK_ID_PREV"]
credit_grouped_cnt = credit_card_balance_cnt.groupby("SK_ID_PREV")
credit_grouped_cnt_max = credit_grouped_cnt.max()
credit_grouped_cnt_max = credit_grouped_cnt_max.add_suffix("_max")
credit_grouped_cnt_max.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CNT_DRAWINGS_ATM_CURRENT_max,CNT_DRAWINGS_CURRENT_max,CNT_DRAWINGS_OTHER_CURRENT_max,CNT_INSTALMENT_MATURE_CUM_max
SK_ID_PREV,,,,
1000018,3.0,15,0.0,4.0
1000030,1.0,14,0.0,5.0
1000031,2.0,4,0.0,10.0
1000035,NaN,0,NaN,0.0
1000077,NaN,0,NaN,0.0


<font size="5"> Because we contract status only show the stages of each transactions in the past applications, it doesn't concern the current application so we leave this feature out

### Summarizing credit_card_balance

In [12]:
credit_card_balance_sk = credit_card_balance.filter(regex = "SK_")
credit_card_balance_sk["SK_ID_PREV"] = credit_card_balance["SK_ID_PREV"]
credit_grouped_sk = credit_card_balance_sk.groupby("SK_ID_PREV")
credit_grouped_sk_mean = credit_grouped_sk.mean()
credit_grouped_sk_mean = credit_grouped_sk_mean.rename({"SK_DPD": "SK_DPD_avg", "SK_DPD_DEF": "SK_DPD_DEF_avg"}, axis = 1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
summarized_credit_balance = credit_grouped_sk_mean.merge(credit_grouped_amt_mean, on  = "SK_ID_PREV")
summarized_credit_balance = summarized_credit_balance.merge(credit_grouped_cnt_max, on = "SK_ID_PREV")
summarized_credit_balance

,SK_ID_CURR,SK_DPD_avg,SK_DPD_DEF_avg,AMT_BALANCE_avg,AMT_CREDIT_LIMIT_ACTUAL_avg,AMT_DRAWINGS_CURRENT_avg,AMT_DRAWINGS_OTHER_CURRENT_avg,AMT_DRAWINGS_POS_CURRENT_avg,AMT_PAYMENT_CURRENT_avg,CNT_DRAWINGS_ATM_CURRENT_max,CNT_DRAWINGS_CURRENT_max,CNT_DRAWINGS_OTHER_CURRENT_max,CNT_INSTALMENT_MATURE_CUM_max
SK_ID_PREV,,,,,,,,,,,,,
1000018,394447.0,0.000000,0.000000,74946.285000,81000.000000,29478.996000,0.0,24078.996000,5541.750000,3.0,15,0.0,4.0
1000030,361282.0,0.000000,0.000000,55991.064375,81562.500000,17257.438125,0.0,19079.929286,6188.631429,1.0,14,0.0,5.0
1000031,131335.0,0.000000,0.000000,52394.439375,149625.000000,28959.615000,0.0,23527.218462,29543.257500,2.0,4,0.0,10.0
1000035,436351.0,0.000000,0.000000,0.000000,225000.000000,0.000000,NaN,NaN,NaN,NaN,0,NaN,0.0
1000077,181153.0,0.000000,0.000000,0.000000,94090.909091,0.000000,NaN,NaN,NaN,NaN,0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2843476,197090.0,15.031579,3.473684,37937.812263,161526.315789,947.368421,0.0,0.000000,3467.420053,2.0,2,0.0,41.0
2843477,168439.0,0.000000,0.000000,1663.076647,15882.352941,688.235294,0.0,0.000000,955.265294,6.0,6,0.0,4.0
2843478,424526.0,0.000000,0.000000,5111.405000,21000.000000,1000.000000,0.0,0.000000,1355.795000,3.0,3,0.0,10.0


In [14]:
summarized_credit_balance.to_csv("./summarized_credit_balance.csv")

## 2.3 Summarizing installments_payments

In [15]:
installments_payments_grouped = installments_payments.groupby("SK_ID_PREV")

In [16]:
description = pd.read_csv("../input/home-credit-default-risk/HomeCredit_columns_description.csv", encoding= "ISO-8859-1")
description = description[description["Table"] == "installments_payments.csv"]
description
def search_description(name):
    return description[description["Row"] == name].values

In [17]:
for i in installments_payments.columns:
    print(search_description(i))

[]
[[215 'installments_payments.csv' 'SK_ID_CURR' 'ID of loan in our sample'
  'hashed']]
[[216 'installments_payments.csv' 'NUM_INSTALMENT_VERSION'
  'Version of installment calendar (0 is for credit card) of previous credit. Change of installment version from month to month signifies that some parameter of payment calendar has changed'
  nan]]
[[217 'installments_payments.csv' 'NUM_INSTALMENT_NUMBER'
  'On which installment we observe payment' nan]]
[[218 'installments_payments.csv' 'DAYS_INSTALMENT'
  'When the installment of previous credit was supposed to be paid (relative to application date of current loan)'
  'time only relative to the application']]
[[219 'installments_payments.csv' 'DAYS_ENTRY_PAYMENT'
  'When was the installments of previous credit paid actually (relative to application date of current loan)'
  'time only relative to the application']]
[[220 'installments_payments.csv' 'AMT_INSTALMENT'
  'What was the prescribed installment amount of previous credit on this 

<font size="5"> Adding balance (AMT_PAYMENT - AMT_INSTALMENT) to see how much the applicants not paying enough and default (True if the balance is negative, False otherwise)

In [18]:
def check_late_payment(arr):
    if arr < 0:
        return 1
    elif arr >= 0:
        return 0
    else:
        return np.nan

In [19]:
installments_payments["DAYS_LATE"] = abs(installments_payments["DAYS_INSTALMENT"]) - abs(installments_payments["DAYS_ENTRY_PAYMENT"])
installments_payments["late_payment"] = installments_payments["DAYS_LATE"].apply(check_late_payment)

In [20]:
installments_payments["balance"] = installments_payments["AMT_PAYMENT"] - installments_payments["AMT_INSTALMENT"]
installments_payments["not_enough_payment"] = installments_payments["balance"].apply(check_late_payment)

In [21]:
installments_payments

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,DAYS_LATE,late_payment,balance,not_enough_payment
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,-7.0,1.0,0.000,0.0
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,0.0,0.0,0.000,0.0
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,0.0,0.0,0.000,0.0
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,-8.0,1.0,0.000,0.0
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,17.0,0.0,-4.455,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,NaN,67.500,NaN,NaN,NaN,NaN,NaN
13605397,1310347,414406,0.0,47,-1539.0,NaN,67.500,NaN,NaN,NaN,NaN,NaN
13605398,1308766,402199,0.0,43,-7.0,NaN,43737.435,NaN,NaN,NaN,NaN,NaN
13605399,1062206,409297,0.0,43,-1986.0,NaN,67.500,NaN,NaN,NaN,NaN,NaN


In [22]:
grouped_installments = installments_payments.groupby("SK_ID_PREV")
total_not_enough_payment = grouped_installments["not_enough_payment"].sum()
total_late_payment = grouped_installments["late_payment"].sum()

In [23]:

installments_payments_grouped = installments_payments.groupby("SK_ID_PREV")
installments_payments_mean = installments_payments_grouped.mean()
installments_payments_mean = installments_payments_mean.add_suffix("_mean")
installments_payments_mean.rename({"SK_ID_CURR_mean": "SK_ID_CURR"}, inplace = True, axis = 1)
installments_payments_mean["CNT_late_payment"] = total_late_payment
installments_payments_mean["CNT_not_enough_payment"] = total_not_enough_payment

In [24]:
installments_payments_mean

,SK_ID_CURR,NUM_INSTALMENT_VERSION_mean,NUM_INSTALMENT_NUMBER_mean,DAYS_INSTALMENT_mean,DAYS_ENTRY_PAYMENT_mean,AMT_INSTALMENT_mean,AMT_PAYMENT_mean,DAYS_LATE_mean,late_payment_mean,balance_mean,not_enough_payment_mean,CNT_late_payment,CNT_not_enough_payment
SK_ID_PREV,,,,,,,,,,,,,
1000001,158271.0,1.500000,1.500000,-253.000000,-269.000000,34221.712500,34221.712500,-16.000000,1.000000,0.000000,0.000000,2.0,0.0
1000002,101962.0,1.250000,2.500000,-1555.000000,-1574.750000,9308.891250,9308.891250,-19.750000,1.000000,0.000000,0.000000,4.0,0.0
1000003,252457.0,1.000000,2.000000,-64.000000,-79.333333,4951.350000,4951.350000,-15.333333,1.000000,0.000000,0.000000,3.0,0.0
1000004,260094.0,1.142857,4.000000,-772.000000,-798.714286,4789.022143,4789.022143,-26.714286,1.000000,0.000000,0.000000,7.0,0.0
1000005,176456.0,1.000000,5.818182,-1543.454545,-1551.909091,14703.210000,13365.609545,-8.454545,0.818182,-1337.600455,0.181818,9.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2843495,260963.0,1.142857,4.000000,-349.000000,-352.857143,113932.883571,113932.883571,-3.857143,0.714286,0.000000,0.000000,5.0,0.0
2843496,425374.0,0.000000,16.235294,-258.058824,-262.058824,9186.311912,8950.975147,-4.000000,0.382353,-235.336765,0.117647,13.0,4.0
2843497,451578.0,1.000000,10.500000,-303.000000,-305.900000,9175.185000,9175.185000,-2.900000,0.650000,0.000000,0.000000,13.0,0.0


In [25]:
installments_payments_mean.to_csv("summarized_installments_payments.csv")

## 2.4 merge installments_and_credit_card

In [26]:
installments_and_credit_card = installments_payments_mean.add_suffix("_installment").merge(summarized_credit_balance.add_suffix("_credit_card"), left_index=True, right_index=True)

In [27]:
installments_and_credit_card["SK_ID_PREV"] = installments_and_credit_card.index
installments_and_credit_card.index = pd.RangeIndex(len(installments_and_credit_card))

In [28]:
installments_and_credit_card["SK_ID_CURR"] = installments_and_credit_card["SK_ID_CURR_installment"]
installments_and_credit_card.drop(["SK_ID_CURR_installment", "SK_ID_CURR_credit_card"], axis = 1)

,NUM_INSTALMENT_VERSION_mean_installment,NUM_INSTALMENT_NUMBER_mean_installment,DAYS_INSTALMENT_mean_installment,DAYS_ENTRY_PAYMENT_mean_installment,AMT_INSTALMENT_mean_installment,AMT_PAYMENT_mean_installment,DAYS_LATE_mean_installment,late_payment_mean_installment,balance_mean_installment,not_enough_payment_mean_installment,...,AMT_DRAWINGS_CURRENT_avg_credit_card,AMT_DRAWINGS_OTHER_CURRENT_avg_credit_card,AMT_DRAWINGS_POS_CURRENT_avg_credit_card,AMT_PAYMENT_CURRENT_avg_credit_card,CNT_DRAWINGS_ATM_CURRENT_max_credit_card,CNT_DRAWINGS_CURRENT_max_credit_card,CNT_DRAWINGS_OTHER_CURRENT_max_credit_card,CNT_INSTALMENT_MATURE_CUM_max_credit_card,SK_ID_PREV,SK_ID_CURR
0,0.0,9.000000,-78.352941,-78.705882,2073.994412,2073.994412,-0.352941,0.235294,0.000000,0.000000,...,29478.996000,0.0,24078.996000,5541.750000,3.0,15,0.0,4.0,1000018,394447.0
1,0.0,10.000000,-115.842105,-118.157895,2284.131316,2284.131316,-2.315789,0.315789,0.000000,0.000000,...,17257.438125,0.0,19079.929286,6188.631429,1.0,14,0.0,5.0,1000030,361282.0
2,0.0,16.500000,-200.593750,-204.437500,11078.721562,11078.721562,-3.843750,0.312500,0.000000,0.000000,...,28959.615000,0.0,23527.218462,29543.257500,2.0,4,0.0,10.0,1000031,131335.0
3,0.0,21.023256,-554.046512,-555.255814,4718.621512,4591.892093,-1.209302,0.162791,-126.729419,0.046512,...,4278.474844,0.0,932.619194,6364.110484,3.0,6,0.0,11.0,1000087,399664.0
4,0.0,49.520000,-1601.760000,-1605.820000,2690.752050,2600.752050,-4.060000,0.400000,-90.000000,0.020000,...,1575.011761,0.0,0.000000,2955.400057,3.0,3,0.0,48.0,1000094,359175.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72461,0.0,49.342857,-1555.180952,-1550.504762,4166.445429,3137.231143,4.676190,0.180952,-1029.214286,0.200000,...,947.368421,0.0,0.000000,3467.420053,2.0,2,0.0,41.0,2843476,197090.0
72462,0.0,45.000000,-1351.516854,-1352.078652,914.213933,914.213933,-0.561798,0.044944,0.000000,0.000000,...,688.235294,0.0,0.000000,955.265294,6.0,6,0.0,4.0,2843477,168439.0
72463,0.0,50.000000,-1490.585859,-1492.020202,1234.057727,1234.057727,-1.434343,0.101010,0.000000,0.000000,...,1000.000000,0.0,0.000000,1355.795000,3.0,3,0.0,10.0,2843478,424526.0
72464,0.0,15.096774,-222.774194,-224.387097,2577.001935,2504.421290,-1.612903,0.258065,-72.580645,0.064516,...,7217.160000,0.0,7732.671429,5625.927692,0.0,2,0.0,13.0,2843493,337804.0


In [29]:
installments_and_credit_card.to_csv("installments_and_credit_card.csv")

In [30]:
print(installments_and_credit_card.shape, summarized_credit_balance.shape, installments_payments_mean.shape)

(72466, 28) (104307, 13) (997752, 13)


# 3. Merge pos_cash

## 3.1 inspect POS_cash

In [31]:
description = pd.read_csv("../input/home-credit-default-risk/HomeCredit_columns_description.csv", encoding= "ISO-8859-1")
description = description[description["Table"] == "POS_CASH_balance.csv"]
description
def search_description(name):
    return description[description["Row"] == name].values

In [32]:
pos_cash = pd.read_csv("../input/home-credit-default-risk/POS_CASH_balance.csv")

In [33]:
pos_cash

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,Active,843,0
10001354,1717234,141565,-19,12.0,0.0,Active,602,0
10001355,1283126,315695,-21,10.0,0.0,Active,609,0
10001356,1082516,450255,-22,12.0,0.0,Active,614,0


In [34]:
search_description("MONTHS_BALANCE")

array([[147, 'POS_CASH_balance.csv', 'MONTHS_BALANCE',
        'Month of balance relative to application date (-1 means the information to the freshest monthly snapshot, 0 means the information at application - often it will be the same as -1 as many banks are not updating the information to Credit Bureau regularly )',
        'time only relative to the application']], dtype=object)

In [35]:
search_description("SK_DPD")

array([[151, 'POS_CASH_balance.csv', 'SK_DPD',
        'DPD (days past due) during the month of previous credit', nan]],
      dtype=object)

In [36]:
search_description("CNT_INSTALMENT_FUTURE")


array([[149, 'POS_CASH_balance.csv', 'CNT_INSTALMENT_FUTURE',
        'Installments left to pay on the previous credit', nan]],
      dtype=object)

In [37]:
grouped_pos = pos_cash.groupby("SK_ID_PREV")

In [38]:
dpd_pos = grouped_pos.agg(SK_DPD_mean = ("SK_DPD", np.mean), SK_DPD_DEF_mean =( "SK_DPD_DEF", np.mean))
dpd_pos["SK_ID_PREV"] = dpd_pos.index
dpd_pos.index = pd.RangeIndex(len(dpd_pos))
dpd_pos

,SK_DPD_mean,SK_DPD_DEF_mean,SK_ID_PREV
0,0.0,0.0,1000001
1,0.0,0.0,1000002
2,0.0,0.0,1000003
3,0.0,0.0,1000004
4,0.0,0.0,1000005
...,...,...,...
936320,0.0,0.0,2843494
936321,0.0,0.0,2843495
936322,0.0,0.0,2843497
936323,0.0,0.0,2843498


## 3.2 Merge pos and previous + merge installments and credit card

In [39]:
pos_and_previous = dpd_pos.add_suffix("_pos").merge(previous_application, left_on = "SK_ID_PREV_pos", right_on = "SK_ID_PREV", how = "right")
pos_and_previous = pos_and_previous.drop(["SK_ID_PREV_pos"], axis = 1)
final = installments_and_credit_card.merge(pos_and_previous, on = "SK_ID_PREV", how = "right")


In [40]:
final = final.drop(['SK_ID_CURR_installment', 'SK_ID_CURR_credit_card', 'SK_ID_CURR_y'], axis = 1)
final = final.rename({"SK_ID_CURR_x": "SK_ID_CURR"}, axis = 1)
final.to_csv("./final_prev.csv")